# Latest Barstool Sports stories

#### Python tools and notebook settings

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import altair as alt
import numpy as np
import datetime as dt
from datetime import date
import re
import time
import random
from tqdm import tqdm
import json

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
month_year_updated = dt.date.today().strftime("%m_%Y")
today = dt.date.today().strftime("%Y-%m-%d")

---

In [5]:
headers = {
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    "Referer": "https://www.gunviolencearchive.org/",
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

#### Snag jsons using pagination in API

In [6]:
data = []

for r in range(1, 80):
    request = requests.get(
        f"https://union.barstoolsports.com/v2/stories/latest?&limit=4000&page={r}",
        headers=headers,
    )
    data.append(pd.DataFrame(request.json()))

In [7]:
stories_df = pd.concat(data)

In [8]:
stories_df.head(1)

,id,brand_id,title,type,url,branch_url,thumbnail,nsfw,comment_count,comment_status_open,category,tag,tags,author,post_type_meta,product,campaign,viewability,is_pinned,is_featured,is_published,is_hidden,is_show_guide,is_membership,is_archive,is_search_indexed,gated_product_id,access_level,ads_disabled,is_featured_section,blocker_description,is_instant_article,site_roughnrowdy,user_id,content_tags,author_is_active,brand,brand_name,is_gold,date,slug,hidden_tags,sites,urls,sharing,updated_at
0,3431718,NaN,JR Smith Says He's Been Blackballed By The NBA Because Of His 2018 Finals Blunder,standard_post,https://www.barstoolsports.com/blog/3431718/jr-smith-says-hes-been-blackballed-by-the-nba-because-of-his-2018-finals-blunder,https://bars.tl/3431718,"{'type': 'small', 'location': 'https://chumley.barstoolsports.com/wp-content/uploads/2019/10/29/', 'images': {'small': 'GettyImages-964676116.jpg?crop=4%3A3%2Csmart', 'medium': 'GettyImages-964676116.jpg?crop=4%3A3%2Csmart', 'large': 'GettyImages-964676116.jpg?crop=4%3A3%2Csmart', 'featured': None}, 'desktop': 'https://chumley.barstoolsports.com/wp-content/uploads/2019/10/29/GettyImages-964676116.jpg?crop=4%3A3%2Csmart&width=560', 'raw': 'https://chumley.barstoolsports.com/wp-content/uploads/2019/10/29/GettyImages-964676116.jpg?crop=4%3A3%2Csmart', 'raw_desktop': 'https://chumley.barstoolsports.com/wp-content/uploads/2019/10/29/GettyImages-964676116.jpg?crop=4%3A3%2Csmart', 'featured': None, 'raw_featured': None}",False,0,True,"[{'id': '97', 'slug': 'boston', 'name': 'Boston'}]",None,"[nba, home, sports, jr-smith, 2018, blackballed, cavs]","{'id': 14607, 'name': 'Greenie', 'author_url': '/bio/14607/greenie/greenie', 'avatar': 'https://chumley.barstoolsports.com/union/2019/10/21/f2e4f8680ef6.png?crop=0.81%2C0.99%2Cx0.04%2Cy0.00&width=64&height=64', 'twitter_handle': 'StoolGreenie', 'has_notifications': True, 'is_active': True}","{'standard_post': {}, 'barstool_original': None, 'podcast': None, 'gallery': None, 'live': None, 'fundraiser': None}",{'id': None},None,0,0,0,1,0,0,0,0,1,None,0,0,0.0,,0,0,uozAMthKasuW3jBvzGPbD8,"[{'metrics': {'published_posts': 12015, 'published_media': 8}, 'tag': 'nba', 'created_at': '2021-03-09T17:17:35.958Z', 'slug': 'nba', 'updated_at': '2022-09-05T15:39:53.648Z', 'visibility': 'public', 'id': '3yRtBqPCQsaGe6evxfCCtn'}, {'metrics': {'published_posts': 2587}, 'tag': 'home', 'visibility': 'public', 'created_at': '2021-05-27T00:42:35.558Z', 'slug': 'home', 'updated_at': '2022-09-05T15:39:53.648Z', 'id': 'a3f882d8-1b73-415b-92ca-4538f1294f45'}, {'metrics': {'published_posts': 28237, 'published_media': 22}, 'tag': 'sports', 'created_at': '2021-03-09T17:17:35.902Z', 'slug': 'sports', 'updated_at': '2022-09-05T15:39:53.648Z', 'visibility': 'public', 'id': 'qt13xJewn1f5pYR7grZdZ2'}, {'metrics': {'published_posts': 81}, 'tag': 'jr smith', 'created_at': '2021-03-09T17:18:12.848Z', 'slug': 'jr-smith', 'updated_at': '2022-09-05T15:39:53.648Z', 'visibility': 'public', 'id': 'dQQSPJAH4wdF15ppurCaz4'}, {'metrics': {'published_posts': 49}, 'tag': '2018', 'created_at': '2021-03-09T17:18:28.357Z', 'slug': '2018', 'updated_at': '2022-09-05T15:39:53.648Z', 'visibility': 'public', 'id': 'dCJkY2MwuSgqQondo78X8D'}, {'tag': 'blackballed', 'visibility': 'public', 'created_at': '2022-09-05T15:39:31.059Z', 'slug': 'blackballed', 'updated_at': '2022-09-05T15:39:53.648Z', 'id': '29a25cbf-573e-429e-aa13-d1afff047f95'}, {'metrics': {'published_posts': 293}, 'tag': 'cavs', 'created_at': '2021-03-09T17:17:45.362Z', 'slug': 'cavs', 'updated_at': '2022-09-05T15:39:53.648Z', 'visibility': 'public', 'id': '1AiQUPrXLgNf9rYNVs2HT1'}]",1.0,None,None,False,2022-09-05T17:15:00.000Z,jr-smith-says-hes-been-blackballed-by-the-nba-because-of-his-2018-finals-blunder,[home],"{'barstoolsports': True, 'barstoolbets': False, 'roughnrowdy': False}","{'barstoolsports': 'https://www.barstoolsports.com/blog/3431718/jr-smith-says-hes-been-blackballed-by-the-nba-because-of-his-2018-finals-blunder', 'barstoolbets': '

In [9]:
stories_df["author_name"] = pd.json_normalize(stories_df["author"])["name"]

In [10]:
stories_df["author_id"] = pd.json_normalize(stories_df["author"])["id"]

In [11]:
stories_df["img_url"] = pd.json_normalize(stories_df["thumbnail"])["desktop"]

#### Slim down the frame

In [12]:
src = stories_df[
    [
        "id",
        "date",
        "type",
        "title",
        "author_name",
        "comment_count",
        "brand_name",
        "category",
        "branch_url",
        "img_url",
        "tags",
    ]
].copy()

In [13]:
src.head()

,id,date,type,title,author_name,comment_count,brand_name,category,branch_url,img_url,tags
0,3431718,2022-09-05T17:15:00.000Z,standard_post,JR Smith Says He's Been Blackballed By The NBA Because Of His 2018 Finals Blunder,Greenie,0,None,"[{'id': '97', 'slug': 'boston', 'name': 'Boston'}]",https://bars.tl/3431718,https://chumley.barstoolsports.com/wp-content/uploads/2019/10/29/GettyImages-964676116.jpg?crop=4%3A3%2Csmart&width=560,"[nba, home, sports, jr-smith, 2018, blackballed, cavs]"
1,3431389,2022-09-05T17:00:00.000Z,standard_post,Fitz's NFL Playoff Predictions Feature Only 4 New Teams Crashing The Party And 2 Defensive Head Coaches In Super Bowl LVII!,Matt Fitzgerald,2,None,"[{'id': '97', 'slug': 'boston', 'name': 'Boston'}]",https://bars.tl/3431389,https://chumley.barstoolsports.com/union/getty/2022/09/03/GettyImages-1364997593.84f065f8.jpeg?crop=4%3A3%2Csmart&width=560,"[sports, nfl, nfl-playoffs]"
2,3431708,2022-09-05T16:45:00.000Z,standard_post,"'I Reacted As A Human Being, As A Man' - Furkan Korkmaz, Explaining How He Fought 3 Georgia Players And Security At EuroBasket",Reags,1,None,"[{'id': '102', 'slug': 'barstoolu', 'name': 'BarstoolU'}]",https://bars.tl/3431708,https://chumley.barstoolsports.com/union/getty/2022/09/05/GettyImages-1242950965.ce2a355c.jpeg?crop=4%3A3%2Csmart&width=560,"[sports, nba, eurobasket, turkey, georgia, furkan-korkmaz]"
3,3431712,2022-09-05T16:30:00.000Z,standard_post,The Best Way To Promote The Next Mission Impossible Movie? Have Tom Cruise Hype It Up While Hanging Out Of a Plane Flying in the Sky,hubbs,2,None,"[{'id': '102', 'slug': 'barstoolu', 'name': 'BarstoolU'}]",https://bars.tl/3431712,https://chumley.barstoolsports.com/union/2022/09/05/Screen-Shot-2022-09-05-at-11.33.17-AM.a2535571.png?crop=0.56%2C0.95%2Cx0.17%2Cy0.01%2Csafe&width=560,"[tom-cruise, movie, plane]"
4,3431715,2022-09-05T16:15:00.000Z,standard_post,Now the Gossip Rags are Saying Tom Brady is 'Sad' Over His 'Serious Disagreement' with Gisele,Jerry Thornton,27,None,"[{'id': '97', 'slug': 'boston', 'name': 'Boston'}]",https://bars.tl/3431715,https://chumley.barstoolsports.com/union/giphy/2022/09/05/487a9c5f.gif?crop=0.73%2C0.99%2Cx0.00%2Cy0.00%2Csafe&width=560,"[tom-brady, gisele, sad, page-six, buccaneers]"


#### Dates/times - convert to Eastern

In [14]:
src["published_date"] = pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern").dt.date
src["published_hour"] = pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern").dt.hour
src["published_day"] = (
    pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern").dt.day_name()
)
src["timestamp"] = pd.to_datetime(src["date"]).dt.tz_convert("US/Eastern")
src.drop("date", axis=1, inplace=True)

In [15]:
src["week_start"] = (
    pd.to_datetime(src["published_date"])
    .dt.to_period("W")
    .apply(lambda r: r.start_time)
)

#### Tags and categories

In [16]:
src["first_tag"] = [l[0] if len(l) > 0 else np.nan for l in src["tags"]]

In [17]:
src[["category_id", "category_slug", "category_name"]] = pd.json_normalize(
    pd.json_normalize(src["category"])[0]
)

In [18]:
src.drop(["category", "category_id", "category_slug"], axis=1, inplace=True)

#### Filter the dataframe to just blog posts

In [19]:
df = src.copy()

In [20]:
df["published_date"].min()

datetime.date(2005, 3, 23)

In [21]:
len(df[df["type"] == "standard_post"])

190226

In [22]:
df.published_date.min()

datetime.date(2005, 3, 23)

---

#### Posts by Billy Football and Rico Bosco

In [23]:
standard_posts = df[df["type"] == "standard_post"]

In [24]:
bf_rb = standard_posts[
    standard_posts["author_name"].str.contains("Billy Football|Rico Bosco")
]

In [25]:
len(bf_rb)

5818

In [26]:
bf_rb[bf_rb["author_name"] == "Rico Bosco"]["published_date"].min()

datetime.date(2011, 10, 24)

In [27]:
bf_rb[bf_rb["author_name"] == "Billy Football"]["published_date"].min()

datetime.date(2010, 2, 3)

#### Post and comment totals for Billy v. Rico: June 2016-June 2022

In [28]:
bf_rb_grouped = (
    bf_rb.groupby(["author_name"])
    .agg({"id": "count", "comment_count": sum})
    .reset_index()
).rename(columns={"id": "posts"})

#### Comments per post

In [29]:
bf_rb_grouped["comments_per_post"] = (
    bf_rb_grouped["comment_count"] / bf_rb_grouped["posts"]
).round(2)

In [30]:
bf_rb_grouped

,author_name,posts,comment_count,comments_per_post
0,Billy Football,3498,48519,13.87
1,Rico Bosco,2320,34278,14.78


---

#### Posts by author

In [31]:
posts_grouped = (
    df[df["type"] == "standard_post"]
    .groupby("author_name")
    .agg({"comment_count": sum, "id": "count"})
    .reset_index()
    .rename(columns={"id": "post_count"})
)

In [32]:
posts_grouped["comments_per_post"] = (
    posts_grouped["comment_count"] / posts_grouped["post_count"]
).round(2)

In [33]:
posts_grouped_freq = posts_grouped[posts_grouped["post_count"] > 10].sort_values(
    "comments_per_post", ascending=False
)

In [34]:
posts_grouped_freq.sort_values("comments_per_post", ascending=False).head()

,author_name,comment_count,post_count,comments_per_post
53,Kayce Smith,1358,49,27.71
9,Biz,881,48,18.35
26,Devlin,6504,375,17.34
103,Will Compton,876,51,17.18
71,Nick,2370,145,16.34


---

## Export

In [35]:
df.to_csv(f"/Users/stiles/data/barstool_{today}.csv", index=False)

In [36]:
df.to_csv("/Users/stiles/data/barstool.csv", index=False)

In [37]:
stories_df.to_json(
    f"/Users/stiles/data/barstool_{today}.json", indent=4, orient="records"
)